In [22]:
# conda install scikit-learn
# install from: http://docs.python-requests.org/en/master/
# to fix ssl installation of python packages issue
# curl https://bootstrap.pypa.io/get-pip.py | python3
# instructions to setup virtualenv
# https://www.tensorflow.org/install/install_mac
# pip3 install -r requirements.txt

import requests
import cv2
import scipy.misc
import numpy as np
import csv
import sklearn.model_selection as model
import tensorflow
from csv import DictReader
from urllib.request import urlopen
from matplotlib.pyplot import imshow
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

learning_rate = 0.001

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
dataset_x = []
dataset_y = []

spamreader = csv.DictReader(open("./datasets/jcpenney_com-ecommerce_sample.csv"))
for row in spamreader:
    if (row["name_title"] != "" and row["product_image_urls"] != ""):
        dataset_x.append(row["product_image_urls"])
        dataset_y.append(row["name_title"])
    
X = np.array(dataset_x)
y = np.array(dataset_y)

X_train, X_test, y_train, y_test = model.train_test_split(X, y, test_size = 0.3)
print(y)

['Alfred Dunner® Essential Pull On Capri Pant'
 'Alfred Dunner® Essential Pull On Capri Pant'
 'Alfred Dunner® Essential Pull On Capri Pant'
 'Alfred Dunner® Essential Pull On Capri Pant'
 'Alfred Dunner® Essential Pull On Capri Pant'
 'Alfred Dunner® Essential Pull On Capri Pant'
 'Alfred Dunner® Feels Like Spring 3/4 Sleeve Leaf Print Top'
 'Alfred Dunner® Feels Like Spring 3/4-Sleeve Leaf Striped Tee - Plus'
 'Alfred Dunner® Feels Like Spring 3/4-Sleeve Watercolor Print Shirt or Pull-On Cargo Capris'
 'Alfred Dunner® Feels Like Spring 3/4-Sleeve Watercolor Print Shirt or Pull-On Cargo Capris'
 'Alfred Dunner® Feels Like Spring 3/4-Sleeve Watercolor Print Shirt or Pull-On Cargo Capris'
 'Alfred Dunner® Feels Like Spring Capris'
 'Alfred Dunner® Feels Like Spring Capris'
 'Alfred Dunner® Feels Like Spring Capris'
 'Alfred Dunner® Feels Like Spring Capris'
 'Alfred Dunner® Feels Like Spring Short-Sleeve Burnout Top and Necklace'
 'Alfred Dunner® Feels Like Spring Short-Sleeve Burnout T

In [17]:
x = tf.placeholder(tf.float32, [None, 24 * 24])
y = tf.placeholder(tf.float32, [None, len(X)])
W1 = tf.Variable(tf.random_normal([5, 5, 1, 64]))
b1 = tf.Variable(tf.random_normal([64]))
W2 = tf.Variable(tf.random_normal([5, 5, 64, 64]))
b2 = tf.Variable(tf.random_normal([64]))
W3 = tf.Variable(tf.random_normal([6*6*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))
W_out = tf.Variable(tf.random_normal([1024, len(X)]))
b_out = tf.Variable(tf.random_normal([len(X)]))

In [18]:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")
    conv_with_b = tf.nn.bias_add(conv, b)
    conv_out = tf.nn.relu(conv_with_b)
    return conv_out

def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding="SAME")

def model(): 
    x_reshaped = tf.reshape(x, shape=[-1, 24, 24, 1])

    conv_out1 = conv_layer(x_reshaped, W1, b1)
    maxpool_out1 = maxpool_layer(conv_out1)
    norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    conv_out2 = conv_layer(norm1, W2, b2)
    norm2 = tf.nn.lrn(conv_out2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    maxpool_out2 = maxpool_layer(norm2)

    maxpool_reshaped = tf.reshape(maxpool_out2, [-1, W3.get_shape().as_list()[0]])
    local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
    local_out = tf.nn.relu(local)

    out = tf.add(tf.matmul(local_out, W_out), b_out)
    return out

In [19]:
image_model = model()
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=image_model, labels=y)
)

# train the model trying to minimize the cost
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# measure the predicter
correct_pred = tf.equal(tf.argmax(image_model), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    onehot_labels = tf.one_hot(y, len(X), on_value=1., off_value=0., axis=-1)
    onehot_vals = sess.run(onehot_labels)
    batch_size = len(dataset_x)
    print("batch size ", batch_size)
    for j in range(0, 1000):
        avg_accuracy_val = 0.
        batch_count = 0.
        
        for i in range(0, len(dataset_x), batch_size):
            batch_data = dataset_x[i:i+batch_size, :]
            batch_onehot_vals = onehot_vals[i:i+batch_size, :]
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: batch_data, y: batch_onehot_vals})
            avg_accuracy_val += accuracy_val
            batch_count += 1.
        
        avg_accuracy_val /= batch_count
        print("Epoch {}. Avg accuracy {}".format(j, avg_accuracy_val))

TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: uint8, int32, int64